# Komunikacja cyfrowa
## Programowanie, lab 02, sztuka programowania: dekompozycja i abstrakcja

<font color='red'>**start: 13:50**</font>

**Motto**
> Istnieją dwa sposoby tworzenia projektu oprogramowania. Jednym ze sposobów jest uczynienie tego tak prostym, że oczywiście nie ma żadnych niedociągnięć. Innym sposobem jest tak skomplikowane, że nie ma oczywistych niedociągnięć. -- C. A. R. Hoare

**Motto 2**

> Design is messy


__Zasada pojedynczej odpowiedzialności__
> (single responsibility principle), zasada mówiąca, że nigdy nie powinno być więcej niż jednego powodu do istnienia podprogramu (za Wikipedią)

## Przykład z wykładu PP
Mamy dany ciąg liczbowy `L` w postaci sekwencji (np. listy). Sprawdź czy istnieją rosnące podciągi ciągu `L`. Podaj je.

```
    [1, 2, 3] -> [1, 2], [2, 3], [1, 2, 3]
    [2, 1, 3, 3] -> [1, 3]
```

Rozwiązanie typowe dla wszystkich początkujących programistów to wrzucenie wszystkich możliwych pętli jedna w drugą i sprawdzanie *on the fly*, m/w tak jak tu

In [4]:
def rosnace_podciagi(L):
    """
    Funkcja zwraca wszystkie rosnące podciągi ciągu L
    """
    ret = []
    for dlugosc in range(2, len(L) + 1):
        for start in range(len(L) - dlugosc + 1):
            rosnacy = True
            for idx in range(start, start + dlugosc - 1):
                if L[idx] > L[idx+1]:
                    rosnacy = False
                    break
            if rosnacy:
                ret.append(L[start:start+dlugosc])
    return ret

In [3]:
for L in ([1, 2, 3], 
          [2, 1, 3], 
          [1, 2, 3, 4], 
          [1, 3, 4, 2], 
          [4, 3, 2, 1]):
    print(L, rosnace_podciagi(L), sep=' -> ')

[1, 2, 3] -> [[1, 2], [2, 3], [1, 2, 3]]
[2, 1, 3] -> [[1, 3]]
[1, 2, 3, 4] -> [[1, 2], [2, 3], [3, 4], [1, 2, 3], [2, 3, 4], [1, 2, 3, 4]]
[1, 3, 4, 2] -> [[1, 3], [3, 4], [1, 3, 4]]
[4, 3, 2, 1] -> []


## Dekompozycja
Tak naprawdę mamy do czynienia z kilkoma problemami, które są rozdzielne

1. podprogram 1: dzielenie listy `L` na podlisty o zadanych długościach, np. z trzyelementowej listy `[1, 2, 3]` możemy stworzyć 2 dwuelementowe podlisty `[1, 2], [2, 3]`
* podprogram 2: sprawdzić czy elementy listy są ułożone w rosnąco (lub jakkolwiek inaczej, malejąco, stale... w zależności od specyfikacji zadania)

Jak już stworzymy powyższe podprogramy (u nas - funkcje), to możemy pokusić się o prosty algorytm

In [12]:
# podprogram 1: dzielenie listy L na podlisty o zadanych długościach
# [100, 200, 300, 400]  ->   [0:0+2], [1:1+2], [2:2+2]
def podziel_na_n_elementowe_podciagi(L, n):
    """
    Funkcja zwraca listę n-elementowych podlist sekwencji liczbowej L.
    """
    ret = []
    for start in range(len(L) - n + 1):  # [0, 1, 2] (3)
        ret.append(L[start:start+n])     # L[4:4+2], L[10:12]
    return ret


print(podziel_na_n_elementowe_podciagi([100, 200, 300, 400], 1))
print(podziel_na_n_elementowe_podciagi([100, 200, 300, 400], 2))
print(podziel_na_n_elementowe_podciagi([100, 200, 300, 400], 3))
print(podziel_na_n_elementowe_podciagi([100, 200, 300, 400], 4))
print(podziel_na_n_elementowe_podciagi([100, 200, 300, 400], 5))

[[100], [200], [300], [400]]
[[100, 200], [200, 300], [300, 400]]
[[100, 200, 300], [200, 300, 400]]
[[100, 200, 300, 400]]
[]


In [22]:
# podprogram 2: sprawdzić czy elementy listy są ułożone w rosnąco
def czy_rosnacy(L):
    """
    Funkcja sprawdza, czy podana sekwencja liczbowa L jest ciągiem rosnącym.
    Zwraca True gdy jest i False w przeciwnym przypadku.
    """
    assert L, 'podaj niepusty L'
    for idx in range(len(L) - 1):
        if L[idx] >= L[idx+1]:
            return False
    return True


print(czy_rosnacy([1, 2, 3]), True)
print(czy_rosnacy([1, 4, 3]), False)
print(czy_rosnacy([1, 4, 13, 13434, 413413413, 131234312423]), True)
# print(czy_rosnacy([]), True) ## !!!

True True
False False
True True


### Algorytm
dane wejściowe: lista liczb `L` o długości `n`

1. podzielić L na podciagi (podlisty) o długości 2, 3, 4, ..., n
2. sprawdzić każdy podciąg
    * jest rosnący?: dodać do zbioru odpowiedzi
    * nie?: odrzucić
3. zwrócić zbiór podciągów (podlist)

In [35]:
def podziel_na_n_elementowe_podciagi(L, n):
    """
    Funkcja zwraca listę n-elementowych podlist sekwencji liczbowej L.
    """
    ret = []
    for start in range(len(L) - n + 1):  # [0, 1, 2] (3)
        ret.append(L[start:start+n])     # L[4:4+2], L[10:12]
    return ret


def czy_rosnacy(L):
    for idx in range(len(L) - 1):
        if L[idx] >= L[idx+1]:
            return False
    return True


def zwroc_wszystkie_podciagi_o_dlugosci_min_2(L):
    '''...'''
    podciagi = []
    for dl in range(2, len(L) + 1):
        p = podziel_na_n_elementowe_podciagi(L, dl)
        podciagi.extend(p)
    return podciagi


def rosnace_ciagi(L):
    '''...'''
    podciagi = zwroc_wszystkie_podciagi_o_dlugosci_min_2(L)

    rosnace = []
    for p in podciagi:
        if czy_rosnacy(p):
            rosnace.append(p)
            
    return rosnace


rosnace_ciagi([1, 2, 5, 4, 5])

[[1, 2], [2, 5], [4, 5], [1, 2, 5]]

### Ćwiczenie 1
Proszę do powyższego zestawu funkcji dopisać 

1. funkcję sprawdzającą czy elementy listy są ułożone malejąco
* funkcję sprawdzającą czy elementy listy są ułożone niemalejąco
* funkcję sprawdzającą czy elementy listy są ułożone nierosnąco
* funkcję sprawdzającą czy elementy listy są ułożone malejąco
* funkcję sprawdzającą czy elementy listy są stałe
* funkcję sprawdzającą czy wartości bezwzględne elementów listy są ułożone malejąco
* funkcję sprawdzającą czy wartości bezwzględne elementów listy są ułożone niemalejąco
* funkcję sprawdzającą czy wartości bezwzględne elementów listy są ułożone nierosnąco
* funkcję sprawdzającą czy wartości bezwzględne elementów listy są ułożone malejąco
* funkcję sprawdzającą czy wartości bezwzględne elementów listy są stałe
* **funkcję sprawdzającą czy wartości bezwzględne elementów listy są naprzemian (mniejszy - większy - mniejszy - większy -...)**
* **funkcję sprawdzającą czy elementy listy są naprzemian (mniejszy - większy - mniejszy - większy -...)**

Każdy student - swoje zadanie, 5 min maks / zadanie. **Wyboldowane** dla ochotnika.

Paulina 1, 6
Marcin 2, 7
Igor 3, 8
Taisja 4, 9
Bartosz 5, 10

In [41]:
# Paulina 1, 6
def czy_malejacy(L):
    for idx in range(len(L) - 1):
        if L[idx] <= L[idx+1]:
            return False
    return True

def czy_wbezwgledne_malejaco(L):
    for idx in range(len(L) - 1):
        if abs(L[idx]) <= abs(L[idx+1]):
            return False
    return True

In [39]:
# Marcin 2, 7
def czy_niemalejacy(L):
    for idx in range(len(L) - 1):
        if L[idx] >= L[idx+1]:
            return False
    return True

def czy_niemalejace_wartosci_bezwzgledne(L):
    ''
    for idx in range(len(L) - 1):
        if abs(L[idx]) >= abs(L[idx+1]):
            return False
    return True

In [47]:
# 3 Igor 3, 8
def czy_nierosnacy(L):
    for idx in range(len(L) - 1):
        if L[idx] >= L[idx+1]:
            return True
    return False

def czy_nierosnacy_bezwzgledne(L):
    for idx in range(len(L) - 1):
        if abs(L[idx]) >= abs(L[idx+1]):
            return True
    return False

czy_nierosnacy([3, 2, 1, 2])

True

In [44]:
# Taisja 4, 9
def czy_malejace(L):
    for idx in range(len(L) - 1):
        if L[idx] <= L[idx+1]:
            return False
    return True

def czy_malejace_bezwględne(L):
    for idx in range(len(L) - 1):
        if abs(L[idx]) <= abs(L[idx+1]):
            return False
    return True

In [43]:
# Bartosz 5, 10
def czy_stale(L):
    for idx in range(len(L) - 1):
        if L[idx] != L[idx + 1]:
            return False
    return True

def czy_war_bezwzgledna_stala(L):
    for idx in range(len(L) - 1):
        if abs(L[idx]) != abs(L[idx + 1]):
            return False
        
    return True

### Ćwiczenie 2
Parafraza głównego problemu. Proszę do powyższego zestawu funkcji dopisać funkcje zwracające wszystkie przynajmniej 2 elementowe podciągi listy `L`

1. ułożone malejąco
* ułożone niemalejąco
* ułożone nierosnąco
* ułożone malejąco
* gdzie elementy są stałe
* gdzie wartości bezwzględne elementów podciągów są ułożone malejąco
* gdzie wartości bezwzględne elementów podciągów są ułożone niemalejąco
* gdzie wartości bezwzględne elementów podciągów są ułożone nierosnąco
* gdzie wartości bezwzględne elementów podciągów są ułożone malejąco
* gdzie wartości bezwzględne elementów podciągów są są stałe
* **gdzie elementy są ułożone naprzemian (mniejszy - większy - mniejszy - większy -...)**
* **gdzie wartości bezwzględne elementów podciągów są są naprzemian (mniejszy - większy - mniejszy - większy-...)**



* 1, 6 Paulina
* 2, 7 Igor
* 3, 8 Taisja
* 4, 9 Bartosz
* 5, 10 Marcin


In [52]:
def podziel_na_n_elementowe_podciagi(L, n):
    """
    Funkcja zwraca listę n-elementowych podlist sekwencji liczbowej L.
    """
    ret = []
    for start in range(len(L) - n + 1):  # [0, 1, 2] (3)
        ret.append(L[start:start+n])     # L[4:4+2], L[10:12]
    return ret

def czy_malejacy(L):
    """
    Funkcja sprawdza, czy podana sekwencja liczbowa L jest ciągiem malejącym.
    Zwraca True gdy jest i False w przeciwnym przypadku.
    """
    assert L, 'podaj niepusty L'
    for idx in range(len(L) - 1):
        if L[idx] <= L[idx+1]:
            return False
    return True

def czy_wbezwgledne_malejaco(L):
    """
    Funkcja sprawdza, czy wartości bezwzględne elementów listy są ułożone malejąco.
    Zwraca True gdy jest i False w przeciwnym przypadku.
    """
    assert L, 'podaj niepusty L'
    for idx in range(len(L) - 1):
        if abs(L[idx]) <= abs(L[idx+1]):
            return False
    return True


def zwroc_wszystkie_podciagi_o_dlugosci_min_2(L):
    '''...'''
    podciagi = []
    for dl in range(2, len(L) + 1):
        p = podziel_na_n_elementowe_podciagi(L, dl)
        podciagi.extend(p)
    return podciagi


def czy_naprzemienne(p):
    '''True gdy P jest naprzemienne, False gdy nie jest'''
    return False


def zad2(L):
    '''...'''
    podciagi = zwroc_wszystkie_podciagi_o_dlugosci_min_2(L)

    malejace = []
    wbmalejaco = []
    rosnace = []
    naprzemienne = []
    for p in podciagi:
        
        if czy_naprzemienne(p):
            naprzemienne.append(p)
        
        if czy_rosnacy(p):
            rosnace.append(p)
            
        if czy_malejacy(p):
            malejace.append(p)
            
        if czy_wbezwgledne_malejaco(p):
            wbmalejaco.append(p)
            
    d = {
        'wbmalejaco': wbmalejaco, 
         'malejace': malejace, 
         'rosnace': rosnace,
         'naprzemienne': naprzemienne
        }
    return d

print(zad2([-5, 6, -4, 3, 2, -1]))

{'wbmalejaco': [[6, -4], [-4, 3], [3, 2], [2, -1], [6, -4, 3], [-4, 3, 2], [3, 2, -1], [6, -4, 3, 2], [-4, 3, 2, -1], [6, -4, 3, 2, -1]], 'malejace': [[6, -4], [3, 2], [2, -1], [3, 2, -1]], 'rosnace': [[-5, 6], [-4, 3]], 'naprzemienne': []}


## Proces budowania aplikacji (w stylu waterfall)

```
__PROBLEM__
           \__Wymagania__
                         \__Analiza__
                                     \__Projekt__
                                                 \__Implementacja__
                                                                   \__ROZWIĄZANIE__
```

Dwa pierwsze to rola klienta, musi powiedzieć co chce. Dwa koleje są kluczowe i najczęściej poświęca się im najwięcej czasu. Dwa ostatnie to w zasadzie jedno i to samo i jest to najłatwiejsza część procesu. Tego typu rozwiązania (waterfall) są wydajne, gdy problem postawiony jest jednoznaczny i posiada ostateczne rozwiązanie, może niekoniecznie jedno, ale ich skończoną ilość. Takie problemy nazywamy zagadnieniami łagodnymi (*Tame problems*). Jeżeli problem wydaje się nie mieć końca, lub możliwości skończonego rozwiązania, to nazywamy go niegodziwym/nikczemnym (*Wicked problem*), przykładowo: tworzenie strony www dla dyrekcji IF...

Wracając do diagramu. Aby móc zaimplementować rozwiązanie, musimy mieć projekt, powstający w oparciu o szczegółową analizę problemu. Jeżeli zaczniemy od dołu (od implementacji) to będziemy musieli często poprawiać ją w nieskończoność...

### Ćwiczenie 3
Napisz program, który będzie **zwracał** `n` kopii ciągu znaków `s`, każda kopia w nowej linii. Jeżeli `s` jest pustym ciągiem znaków, to `s = 'Hello world'`.


In [59]:
# Igor
def ciagi_znakow(ciag_znakow, ilosc_powtorzen):
    if ciag_znakow == '':
        ciag_znakow = "Hello world"
    for ilosc in range(ilosc_powtorzen):
        print(ciag_znakow)
        
ciagi_znakow("", 3), ciagi_znakow("Kola", 3)

Hello world
Hello world
Hello world
Kola
Kola
Kola


(None, None)

In [60]:
# Marcin
def n_kopii_ciagu_znakow(L, n):
    ''
    i = 0
    while i < n:
        i += 1
        print(L)
        
n_kopii_ciagu_znakow([1, 2, 3, 4, 5, 6, 7], 5)
n_kopii_ciagu_znakow("", 5)  #???

[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]







In [66]:
# Bartosz
def mnozenie_stringa(s, n):
    ret = ''
    
    if s == '':
        s = 'Hello world'
    
    else:
        for num in range(n + 1):
            ret += s + '\n'
        return ret

In [67]:
print(mnozenie_stringa("Ala", 10))
print(mnozenie_stringa("", 10))

Ala
Ala
Ala
Ala
Ala
Ala
Ala
Ala
Ala
Ala
Ala

None


In [68]:
def hello_str_n(s, n):
    "..."
    if s == "":
        return "Hello World" * n
    else:
        return s * n
    
print(hello_str_n("", 3))
print(hello_str_n("Ala ma kota", 3))

Hello WorldHello WorldHello World
Ala ma kotaAla ma kotaAla ma kota


In [71]:
def hello_str_n_v2(s, n):
    "..."
    if s:
        ret = s
    else:
        ret = "Hello World"
    
    for i in range(n):
        print(ret)
        
    
print(hello_str_n_v2("", 3))
print(hello_str_n_v2("Ala ma kota", 3))

Hello World
Hello World
Hello World
None
Ala ma kota
Ala ma kota
Ala ma kota
None


In [72]:
def hello_str_n_v3(s, n=1):
    "..."
    if not s:
        tekst = "Hello World"
    else:
        tekst = s
    
    ret = ""
    for i in range(n):
        ret += str(i) + " " + tekst + "\n"
        
    return ret
    
r = hello_str_n_v3("", 3)
print(r)

print(hello_str_n_v3("Ala ma kota", 3))

0 Hello World
1 Hello World
2 Hello World

0 Ala ma kota
1 Ala ma kota
2 Ala ma kota



### Ćwiczenie 4
Napisz program, który będzie zwracał `n` kopii ciągu znaków `s`, każda kopia w nowej linii. Jeżeli `s` jest pustym ciągiem znaków, to `s = 'Hello world'`.
Zaimplementuj rozwiązanie w/g projektu podanego poniżej.

In [80]:
def hello_lub_string(s):
    return s if s else 'Hello world'

def zwroc_tekst_n_razy(tekst, n):
    wyjscie = ''
    for ilosc in range(n):
        wyjscie += tekst + '\n'
    return wyjscie

def str_n(s, n=1):
    '''
    Funkcja zwraca string s n razy. 
    Jezeli s jest puste, zwraca n razy "Hello World"
    '''
    tekst = hello_lub_string(s)
    ret = zwroc_tekst_n_razy(tekst, n)
    return ret

In [82]:
print(str_n("", 3))
print(str_n("Palindrom", 3))

Hello world
Hello world
Hello world

Palindrom
Palindrom
Palindrom



### Ćwiczenie 5 Znów palindrom
Zaprojektuj program sprawdzający czy dany ciąg znaków jest palindromem, tak by dowolne palindromy mogły być sprawdzane. 

* `czy_palindrom('kajak') -> True`
* `czy_palindrom('Kajak.') -> True`
* `czy_palindrom('Kobyła ma za mały bok!') -> True`
* `czy_palindrom('Może jutro ta dama da tortu jeżom...') -> True`
* `czy_palindrom('Madam, I'm Adam.') -> True`
* `czy_palindrom('A wart wór kota? To krów trawa!') -> True`
* `czy_palindrom('O, ty z Katowic, Iwo? Tak, Zyto.') -> True`

Projekt
1. pozbyć się wszystkiego oprócz liter
2. zamienić wszystkie duże literki na małe (lub odwrotnie)
3. sprawdzić czy palindrom
4. zwrócić True/False

In [86]:
def zwroc_tylko_litery(s):
    return ''.join([el for el in s if el.isalpha()])

def tylko_male_litery(s):
    return s.lower()

def czy_palindrom(s):
    return s == s[::-1]

def sprawdz_czy_palindrom(s):
    s = tylko_male_litery(s)
    s = zwroc_tylko_litery(s)
    ret = czy_palindrom(s)
    return ret

sprawdz_czy_palindrom('O, ty z Katowic, Iwo? Tak, Zyto.')

True

### Ćwiczenie 6 Śmiertelne króliki Fibonacciego

Leonardo Fibonacci opracował swój słynny ciąg modelując liczebność populacji królików. Zaczynamy od jednej pary królików, która po miesiącu dojrzewa i po następnym miesiącu wydaje parę młodych i natychmiast ponownie zachodzi w ciążę, po następnym miesiącu pierwsza para wydaje kolejne młode a druga dojrzewa i zachodzi w ciążę itd. Wielkość populacji (liczba par) w kolejnych miesiącach wynosi 1, 1, 2, 3, 5, 8... Model ten zakłada, że każdej parze zawsze rodzą się dokładnie jeden samiec i jedna samica, ale głównym problemem w modelu jest to, że króliki są nieśmiertelne!

![Image of rabbits](http://www.drawinghowtodraw.com/drawing-lessons/drawing-animals-creatures-lessons/images/howtodrawrabbiesbunniesdrawinglessons_html_37f16756.png)

Urealnijmy trochę ten problem. Napisz funkcję obliczającą liczebność populacji śmiertelnych królików. Funkcja przyjmuje trzy argumenty - ``miesiąc``, dla którego obliczamy liczebność, ``miot`` - liczba par, które się rodzą jednorazowo z jednej pary oraz ``zycie`` - czas życia królików (w miesiącach). Minimalna długość życia królików to 3 miesiące. Oznacza to, że każda para zdąży się rozmnożyć przynajmniej 2 razy. W ostatnim miesiącu życia nastąpi **najpierw poród, a dopiero potem śmierć** pary królików.

Jak teraz wyglądać będzie populacja króli po czasie jaki zadajemy w parametrze ``miesiac``? Rozwiązanie zaprojektuj z użyciem abstrakcji oraz dekompozycji. Rozbij zadanie na podproblemy i do każdego zbuduj funkcję. Na koniec wykorzystaj podprogramy do odpowiedzi na pytanie Pana Leonarda: ile to będzie tych króli po `n` miesiącach?

__Specyfikacja__

Wejście
* miesiac: INT > 0
* miot: INT > 0
* zycie: INT >= 3

Wyjście
* populacja: INT